# Lesson 3: Reflection and Blogpost Writing

## Setup

In [15]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [16]:
task = '''
        Write a concise but engaging blogpost about
       space laser network. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [17]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [18]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [19]:
print(reply)

Title: "Revolutionizing Communication: The Rise of Space Laser Networks"

In the vast expanse of space, a new era of connectivity is emerging through space laser networks. These networks use beams of light to transmit data between satellites, enabling faster and more secure communication. With the potential to revolutionize industries such as telecommunications and space exploration, space laser networks offer unprecedented speeds and reliability. Imagine streaming high-definition videos from anywhere on Earth or beyond without delays – all made possible by this cutting-edge technology. The future of communication is indeed looking bright with the development of space laser networks.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [20]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [21]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       space laser network. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Revolutionizing Communication: The Rise of Space Laser Networks"

In the vast expanse of space, a new era of connectivity is emerging through space laser networks. These networks use beams of light to transmit data between satellites, enabling faster and more secure communication. With the potential to revolutionize industries such as telecommunications and space exploration, space laser networks offer unprecedented speeds and reliability. Imagine streaming high-definition videos from anywhere on Earth or beyond without delays – all made possible by this cutting-edge technology. The future of communication is indeed looking bright with the development of space laser networks.

------------------------------------------

## Nested chat

In [22]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [23]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [24]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [25]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [26]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [27]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [28]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       space laser network. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Revolutionizing Communication: The Rise of Space Laser Networks"

In the vast expanse of space, a new era of connectivity is emerging through space laser networks. These networks use beams of light to transmit data between satellites, enabling faster and more secure communication. With the potential to revolutionize industries such as telecommunications and space exploration, space laser networks offer unprecedented speeds and reliability. Imagine streaming high-definition videos from anywhere on Earth or beyond without delays – all made possible by this cutting-edge technology. The future of communication is indeed looking bright with the development of space laser networks.

------------------------------------------


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

Aggregating the feedback from all reviewers, it is clear that the content on space laser networks is well-written and engaging. However, there are some key areas that need to be addressed:

1. Incorporate relevant keywords for SEO optimization.
2. Optimize the meta description for better search engine performance.
3. Add a clear call-to-action for improved user engagement.
4. Ensure that all claims about the benefits and capabilities of space laser networks are accurately supported by reliable sources to avoid misleading the audience.
5. Check for any potential intellectual property concerns related to the technology or terminology used in the article.
6. Verify that no sensitive or classified information about space communication technology is disclosed to prevent security or confidentiality breaches.

Incorporating these suggestions will not only enhance the SEO performance a

## Get the summary

In [29]:
print(res.summary)

Title: "Revolutionizing Communication: The Rise of Space Laser Networks"

In the expanse of space, a new frontier emerges with space laser networks. These cutting-edge networks revolutionize communication through light beams, promising rapid and secure data transmission. Industries like telecommunications and space exploration stand to benefit greatly from this technology's speed and reliability. Picture seamless high-definition video streaming from any location, whether on Earth or beyond. Backed by reliable sources, the future looks bright with space laser networks. Optimize your connectivity and embrace the future today.
